In [3]:
import pandas as pd
import requests
import time

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [54]:
!pip install python-dotenv

In [ ]:
import pandas as pd
import requests
import time

In [55]:
from dotenv import load_dotenv
import os

load_dotenv()
FS_API_KEY = os.getenv("FS_API_KEY")

bike_stations_philly = pd.read_csv("../data/philly.csv")
bike_stations_philly

# Step 2: Prepare the API request
url = "https://places-api.foursquare.com/places/search"
headers = {
    "Authorization": f"Bearer {FS_API_KEY}",
    "X-Places-API-Version": "2025-06-17"
}

all_results = []

# Step 3: Loop over each station
for i, row in bike_stations_philly.iterrows():
    lat, lon = row['latitude'], row['longitude']
    params = {
        "ll": f"{lat},{lon}",
        "radius": 1000,
        "limit": 10
    }
    
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    
    for place in data.get("results", []):
        place["station_name"] = row["name"]
        all_results.append(place)
        

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [28]:
POI_results = []

for place in all_results:
    categories = place.get("categories")
    if categories and len(categories) > 0:
        category_name = categories[0].get("name", "Unknown")
    else:
        category_name = "Unknown"
    
    parsed = {
        "station_name": place.get("station_name"),
        "place_name": place.get("name"),
        "latitude": place.get("latitude"),
        "longitude": place.get("longitude"),
        "address": place.get("location", {}).get("formatted_address"),
        "distance_m": place.get("distance"),
        "category": category_name
    }
    POI_results.append(parsed)

Put your parsed results into a DataFrame

In [29]:
phi_stn_poi = pd.DataFrame(POI_results)
phi_stn_poi.head()

,station_name,place_name,latitude,longitude,address,distance_m,category
0,Ridge & Heritage,Schuykill River Trail,40.008748,-75.195214,"Philadelphia, PA 19127",223,Hiking Trail
1,Ridge & Heritage,Dunkin',40.009403,-75.196519,"4268 Ridge Ave, Philadelphia, PA 19129",99,Donut Shop
2,Ridge & Heritage,Thunder Mug Cafe,40.009411,-75.194689,"4233 Ridge Ave, Philadelphia, PA 19129",216,Coffee Shop
3,Ridge & Heritage,Shan Chuan,40.009166,-75.194114,"4211 Ridge Ave (Midvale Ave.), Philadelphia, P...",272,Chinese Restaurant
4,Ridge & Heritage,LeBus East Falls,40.008862,-75.193664,"4201 Ridge Ave (Indian Queen Lane), Philadelph...",321,New American Restaurant


In [47]:
foodbev_philly = phi_stn_poi[phi_stn_poi["category"].str.contains("restaurant|café|coffee|bar", case=False, na=False)]

In [48]:
foodbev_philly.head()

,station_name,place_name,latitude,longitude,address,distance_m,category
2,Ridge & Heritage,Thunder Mug Cafe,40.009411,-75.194689,"4233 Ridge Ave, Philadelphia, PA 19129",216,Coffee Shop
3,Ridge & Heritage,Shan Chuan,40.009166,-75.194114,"4211 Ridge Ave (Midvale Ave.), Philadelphia, P...",272,Chinese Restaurant
4,Ridge & Heritage,LeBus East Falls,40.008862,-75.193664,"4201 Ridge Ave (Indian Queen Lane), Philadelph...",321,New American Restaurant
20,Huntingdon & Collins,Cook and Shaker,39.982695,-75.120702,"2301 E Albert St (at Tulip St), Philadelphia, ...",310,American Restaurant
24,Huntingdon & Collins,Atlantis: The Lost Bar,39.982340,-75.126862,"2442 Frankford Ave, Philadelphia, PA 19125",375,Bar


In [52]:
foodbev_philly.to_csv("fs_phi_poi_clean.csv")
phi_stn_poi.to_csv("fs_phi_poi_raw.csv")

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating